# Direct Radio Access to Test Byte Files

In [1]:
from __future__ import print_function
from builtins import *                  # python3 types
from time import sleep
import struct as pystruct
from binascii import hexlify
import sys
import os.path
import datetime

In [2]:
sys.path.insert(0,"../tagnet")
from tagnet.tagmessages import TagMessage, TagPoll, TagGet, TagPut, TagDelete, TagHead
from tagnet.tagnames import TagName
from tagnet.tagtlv import TagTlv, TagTlvList, tlv_types

('tagnet: ', '/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py', '/mnt/Open/TagNet/notebooks')
tagnet driver version 0.0.25


In [3]:
sys.path.insert(0,"../tagfuse")
from tagfuse.radioutils import payload2values, radio_receive_msg, radio_send_msg
from tagfuse.radioutils import radio_start, radio_config, radio_show_config

('si446x version: ', '0.2.6')


In [4]:
from tagfuse.tagfuseargs import __version__
'tagfuse', __version__

('tagfuse', '0.0.33')

In [5]:
print('Test Start Time: {}'.format(datetime.datetime.now()))
#print('Si446x Radio Device Driver Version: {}'.format(si446x_device_version()))

Test Start Time: 2018-03-28 22:21:17.296551


##  Start up Radio

In [6]:
radio = radio_start()

In [28]:
radio_show_config(radio.dump_radio())


int_ctl_group_s: 4
073b 2328 
 enable:  CHIP_INT_STATUS_EN MODEM_INT_STATUS_EN PH_INT_STATUS_EN ph_pend:  PACKET_SENT PACKET_RX CRC_ERROR TX_FIFO_ALMOST_EMPTY RX_FIFO_ALMOST_FULL modem_pend:  INVALID_SYNC PREAMBLE_DETECT SYNC_DETECT chip_pend:  FIFO_UNDERFLOW_OVERFLOW_ERROR CMD_ERROR

global_group_s: 10
5200 1860 0000 0160 0000 
Container:
    xo_tune = 82
    clk_cfg = Container:
        divided_clk_en = 'DISABLE'
        divided_clk_sel = 'DIV_1'
        clk_32k_sel = 'OFF'
    low_batt_thresh = 24
    global_config = Container:
        sequencer_mode = 'FAST'
        fifo_mode = 'SPLIT_FIFO'
        protocol = 'GENERIC'
        power_mode = 'HIGH_PERF'
    wut_config = Container:
        wut_ldc_enable = 'DISABLE'
        wut_cal_period = '_1_SEC'
        wut_lbd_en = 'DISABLE'
        wut_en = 'DISABLE'
        cal_en = 'DISABLE'
    wut_m = 1
    wut_r = Container:
        RESERVED_WRITE_ONE = 'ONE'
        wut_sleep = 'SLEEP'
        wut_r_ = 0
    wut_ldc = 0
    wut_cal = 0

s

### Set Defaults

In [51]:
# default paramters
MAX_WAIT            = 1
MAX_RECV            = 255
MAX_PAYLOAD         = 254
MAX_RETRIES         = 10
RADIO_POWER         = 10
SHORT_DELAY         = 1

### Read Test Bytes 

In [9]:
STOP


NameError: name 'STOP' is not defined

In [ ]:
TestByteName = 'echo'

In [ ]:
test_byte_name = TagName([TagTlv(tlv_types.NODE_ID, -1),
                            TagTlv('tag'),
                            TagTlv('.test'),
                            TagTlv(TestByteName),
                            TagTlv('byte'),
                            TagTlv(tlv_types.OFFSET, 1),
                            TagTlv(tlv_types.SIZE, 200),
                             ])

In [ ]:
req_obj = TagGet(test_byte_name)
#    print(get_gps_xyz.name)
req_msg = req_obj.build()

In [ ]:
len(req_msg), req_obj.header, req_obj.name, req_obj.payload

In [ ]:
sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)
rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
len(rsp_msg), rssi

In [ ]:
if rsp_msg:
    #        print(hexlify(rsp_msg))
    rsp_obj = TagMessage(rsp_msg)
    #        print(rsp_obj.header.options.param.error_code)
    #        print(rsp_obj.payload)
    if rsp_obj.payload:
        print(len(rsp_msg), rsp_obj.header, rsp_obj.name, rsp_obj.payload)

In [ ]:
rsp_obj.payload[1].value()

In [ ]:
sstatus, rstatus

In [ ]:
sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)

In [ ]:
rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
rsp_msg, rssi

In [ ]:
for x in range(10):
    print(datetime.datetime.now())
    sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)
    #print(sstatus)
    print(datetime.datetime.now())
    rsp_msg, rssi, status = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
    print(rssi, len(rsp_msg))
    print(rstatus)

In [ ]:
for z in range(10000):
    for RADIO_POWER in [1,10,20,30]:
        for i in range(10):
            rstatus = radio_send_msg(radio, req_msg, RADIO_POWER);
            rsp_msg, rssi, sstatus = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
            if rssi == -1:
                break
            rsp_obj = TagMessage(rsp_msg)
            print(rsp_obj.payload)

In [ ]:
radio.spi.trace.display(radio.spi.trace.filter(count=-10))

### Write Test Bytes 

In [ ]:
STOP


In [ ]:
def test_write_bytes(TestByteName, buf):
    name = TagName([TagTlv(tlv_types.NODE_ID, -1),
                          TagTlv('tag'),
                          TagTlv('.test'),
                          TagTlv(TestByteName),
                          TagTlv('byte'),
                          TagTlv(tlv_types.OFFSET, 0),
                          TagTlv(tlv_types.SIZE, len(buf)),
                             ])
    req_obj = TagPut(name, pl=buf)
    req_msg = req_obj.build()
    sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)
    rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, 3)
    if rsp_msg:
        #        print(hexlify(rsp_msg))
        rsp_obj = TagMessage(rsp_msg)
        #        print(rsp_obj.header.options.param.error_code)
        #        print(rsp_obj.payload)
        # print(rsp_obj.header, rsp_obj.name)
        if rsp_obj.payload:
            return rsp_obj.payload[0].value(), rssi, sstatus, rstatus
        return none, rssi, sstatus, rstatus
    return None, None, sstatus, rstatus

In [ ]:
amt, rssi, sstatus, rstatus = test_write_bytes('echo', bytearray('0123456789' * 20))

In [ ]:
'send', sstatus, 'receive', rstatus

In [ ]:
sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)

In [ ]:
sstatus

In [ ]:
rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
rsp_msg, rssi

In [ ]:
radio.trace.display(radio.trace.filter(count=-10))

In [ ]:
a,b,c = sstatus[6:9]

In [ ]:
a.items()[0][1]

## Get Tag's RSSI value

In [ ]:
STOP

In [14]:
def tag_get_rssi():
    name = TagName([TagTlv(tlv_types.NODE_ID, -1),
                            TagTlv('tag'),
                            TagTlv('.test'),
                            TagTlv('rssi'),
                             ])
    req_obj = TagGet(name)
    req_msg = req_obj.build()
    sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)
    rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, 3)
    if rsp_msg:
        #        print(hexlify(rsp_msg))
        rsp_obj = TagMessage(rsp_msg)
        #        print(rsp_obj.header.options.param.error_code)
        #        print(rsp_obj.payload)
        # print(rsp_obj.header, rsp_obj.name)
        if rsp_obj.payload:
            return rsp_obj.payload[0].value(), rssi, sstatus, rstatus
        return none, rssi, sstatus, rstatus
    return None, None, sstatus, rstatus

In [15]:
tag_get_rssi()

(201,
 242,
 [datetime.datetime(2018, 3, 28, 22, 22, 26, 557168),
  'c',
  '64',
  ',',
  '30',
  'w',
  Container({'CHIP_READY': True, 'STATE_CHANGE': True, 'WUT': False, 'CMD_ERROR': False, 'FIFO_UNDERFLOW_OVERFLOW_ERROR': False, 'LOW_BATT': False, 'CAL': False}),
  Container({'INVALID_SYNC': False, 'SYNC_DETECT': False, 'POSTAMBLE_DETECT': False, 'PREAMBLE_DETECT': True, 'INVALID_PREAMBLE': False, 'RSSI': False, 'RSSI_JUMP': False}),
  datetime.datetime(2018, 3, 28, 22, 22, 26, 577439),
  'w',
  Container({'CHIP_READY': True, 'STATE_CHANGE': False, 'WUT': False, 'CMD_ERROR': False, 'FIFO_UNDERFLOW_OVERFLOW_ERROR': False, 'LOW_BATT': False, 'CAL': False}),
  Container({'INVALID_SYNC': False, 'SYNC_DETECT': False, 'POSTAMBLE_DETECT': False, 'PREAMBLE_DETECT': True, 'INVALID_PREAMBLE': False, 'RSSI': False, 'RSSI_JUMP': False}),
  datetime.datetime(2018, 3, 28, 22, 22, 26, 583226),
  'w',
  Container({'CHIP_READY': True, 'STATE_CHANGE': False, 'WUT': False, 'CMD_ERROR': False, 'FIFO_UN

In [50]:
RADIO_POWER=2
for z in range(100):
    for RADIO_POWER in [1,10,20,30]:
        for i in range(10):
            remote_rssi, rssi, _, _ = tag_get_rssi()
            if rssi == -1:
                break
            print('{:>4}.{:<4} power: {:3},   rssi l/r:  {:4>}/{:<4}'.format(z,i, RADIO_POWER, rssi, remote_rssi))

   0.0    power:   1,   rssi l/r:  218/157 
   0.1    power:   1,   rssi l/r:  218/157 
   0.2    power:   1,   rssi l/r:  218/157 
   0.3    power:   1,   rssi l/r:  218/157 
   0.4    power:   1,   rssi l/r:  218/157 
   0.5    power:   1,   rssi l/r:  218/157 
   0.6    power:   1,   rssi l/r:  218/157 
   0.7    power:   1,   rssi l/r:  218/157 
   0.8    power:   1,   rssi l/r:  218/157 
   0.9    power:   1,   rssi l/r:  218/157 
   0.0    power:  10,   rssi l/r:  218/201 
   0.1    power:  10,   rssi l/r:  218/201 
   0.2    power:  10,   rssi l/r:  218/201 
   0.3    power:  10,   rssi l/r:  218/201 
   0.4    power:  10,   rssi l/r:  218/201 
   0.5    power:  10,   rssi l/r:  218/201 
   0.6    power:  10,   rssi l/r:  218/201 
   0.7    power:  10,   rssi l/r:  218/201 
   0.8    power:  10,   rssi l/r:  218/201 
   0.9    power:  10,   rssi l/r:  218/201 
   0.0    power:  20,   rssi l/r:  218/214 
   0.1    power:  20,   rssi l/r:  218/214 
   0.2    power:  20,   rssi l/r

   4.8    power:  20,   rssi l/r:  218/214 
   4.9    power:  20,   rssi l/r:  218/214 
   4.0    power:  30,   rssi l/r:  218/222 
   4.1    power:  30,   rssi l/r:  218/222 
   4.2    power:  30,   rssi l/r:  218/222 
   4.3    power:  30,   rssi l/r:  218/222 
   4.4    power:  30,   rssi l/r:  218/222 
   4.5    power:  30,   rssi l/r:  218/222 
   4.6    power:  30,   rssi l/r:  218/222 
   4.7    power:  30,   rssi l/r:  218/222 
   4.8    power:  30,   rssi l/r:  218/222 
   4.9    power:  30,   rssi l/r:  218/222 
   5.0    power:   1,   rssi l/r:  218/157 
   5.1    power:   1,   rssi l/r:  218/157 
   5.2    power:   1,   rssi l/r:  218/157 
   5.3    power:   1,   rssi l/r:  218/157 
   5.4    power:   1,   rssi l/r:  218/157 
   5.5    power:   1,   rssi l/r:  218/157 
   5.6    power:   1,   rssi l/r:  218/157 
   5.7    power:   1,   rssi l/r:  218/157 
   5.8    power:   1,   rssi l/r:  218/157 
   5.9    power:   1,   rssi l/r:  218/157 
   5.0    power:  10,   rssi l/r

   9.6    power:  10,   rssi l/r:  218/201 
   9.7    power:  10,   rssi l/r:  218/201 
   9.8    power:  10,   rssi l/r:  218/201 
   9.9    power:  10,   rssi l/r:  218/201 
   9.0    power:  20,   rssi l/r:  218/214 
   9.1    power:  20,   rssi l/r:  218/214 
   9.2    power:  20,   rssi l/r:  218/214 
   9.3    power:  20,   rssi l/r:  218/214 
   9.4    power:  20,   rssi l/r:  218/214 
   9.5    power:  20,   rssi l/r:  218/214 
   9.6    power:  20,   rssi l/r:  218/214 
   9.7    power:  20,   rssi l/r:  218/214 
   9.8    power:  20,   rssi l/r:  218/214 
   9.9    power:  20,   rssi l/r:  218/214 
   9.0    power:  30,   rssi l/r:  218/221 
   9.1    power:  30,   rssi l/r:  218/221 
   9.2    power:  30,   rssi l/r:  218/221 
   9.3    power:  30,   rssi l/r:  218/221 
   9.4    power:  30,   rssi l/r:  218/221 
   9.5    power:  30,   rssi l/r:  218/221 
   9.6    power:  30,   rssi l/r:  218/221 
   9.7    power:  30,   rssi l/r:  218/221 
   9.8    power:  30,   rssi l/r

  14.4    power:   1,   rssi l/r:  218/157 
  14.5    power:   1,   rssi l/r:  218/157 
  14.6    power:   1,   rssi l/r:  218/157 
  14.7    power:   1,   rssi l/r:  218/157 
  14.8    power:   1,   rssi l/r:  218/156 
  14.9    power:   1,   rssi l/r:  218/157 
  14.0    power:  10,   rssi l/r:  218/201 
  14.1    power:  10,   rssi l/r:  218/201 
  14.2    power:  10,   rssi l/r:  218/201 
  14.3    power:  10,   rssi l/r:  218/201 
  14.4    power:  10,   rssi l/r:  218/201 
  14.5    power:  10,   rssi l/r:  218/201 
  14.6    power:  10,   rssi l/r:  218/201 
  14.7    power:  10,   rssi l/r:  218/201 
  14.8    power:  10,   rssi l/r:  218/201 
  14.9    power:  10,   rssi l/r:  218/201 
  14.0    power:  20,   rssi l/r:  218/215 
  14.1    power:  20,   rssi l/r:  218/214 
  14.2    power:  20,   rssi l/r:  218/214 
  14.3    power:  20,   rssi l/r:  218/214 
  14.4    power:  20,   rssi l/r:  218/214 
  14.5    power:  20,   rssi l/r:  218/214 
  14.6    power:  20,   rssi l/r

  18.2    power:  30,   rssi l/r:  218/221 
  18.3    power:  30,   rssi l/r:  218/222 
  18.4    power:  30,   rssi l/r:  218/222 
  18.5    power:  30,   rssi l/r:  218/222 
  18.6    power:  30,   rssi l/r:  218/222 
  18.7    power:  30,   rssi l/r:  218/222 
  18.8    power:  30,   rssi l/r:  218/222 
  18.9    power:  30,   rssi l/r:  218/222 
  19.0    power:   1,   rssi l/r:  218/157 
  19.1    power:   1,   rssi l/r:  218/157 
  19.2    power:   1,   rssi l/r:  218/157 
  19.3    power:   1,   rssi l/r:  218/157 
  19.4    power:   1,   rssi l/r:  218/157 
  19.5    power:   1,   rssi l/r:  218/157 
  19.6    power:   1,   rssi l/r:  218/157 
  19.7    power:   1,   rssi l/r:  218/157 
  19.8    power:   1,   rssi l/r:  218/157 
  19.9    power:   1,   rssi l/r:  218/157 
  19.0    power:  10,   rssi l/r:  218/201 
  19.1    power:  10,   rssi l/r:  218/201 
  19.2    power:  10,   rssi l/r:  218/201 
  19.3    power:  10,   rssi l/r:  218/201 
  19.4    power:  10,   rssi l/r

  23.0    power:  20,   rssi l/r:  218/214 
  23.1    power:  20,   rssi l/r:  218/214 
  23.2    power:  20,   rssi l/r:  218/214 
  23.3    power:  20,   rssi l/r:  218/214 
  23.4    power:  20,   rssi l/r:  218/214 
  23.5    power:  20,   rssi l/r:  218/214 
  23.6    power:  20,   rssi l/r:  218/214 
  23.7    power:  20,   rssi l/r:  218/214 
  23.8    power:  20,   rssi l/r:  218/214 
  23.9    power:  20,   rssi l/r:  218/214 
  23.0    power:  30,   rssi l/r:  218/222 
  23.1    power:  30,   rssi l/r:  218/221 
  23.2    power:  30,   rssi l/r:  218/221 
  23.3    power:  30,   rssi l/r:  218/221 
  23.4    power:  30,   rssi l/r:  218/221 
  23.5    power:  30,   rssi l/r:  218/222 
  23.6    power:  30,   rssi l/r:  218/222 
  23.7    power:  30,   rssi l/r:  218/222 
  23.8    power:  30,   rssi l/r:  218/221 
  23.9    power:  30,   rssi l/r:  218/222 
  24.0    power:   1,   rssi l/r:  218/157 
  24.1    power:   1,   rssi l/r:  218/156 
  24.2    power:   1,   rssi l/r

  28.8    power:   1,   rssi l/r:  218/156 
  28.9    power:   1,   rssi l/r:  218/156 
  28.0    power:  10,   rssi l/r:  218/201 
  28.1    power:  10,   rssi l/r:  218/201 
  28.2    power:  10,   rssi l/r:  218/201 
  28.3    power:  10,   rssi l/r:  218/201 
  28.4    power:  10,   rssi l/r:  218/201 
  28.5    power:  10,   rssi l/r:  218/201 
  28.6    power:  10,   rssi l/r:  218/201 
  28.7    power:  10,   rssi l/r:  218/201 
  28.8    power:  10,   rssi l/r:  218/201 
  28.9    power:  10,   rssi l/r:  218/201 
  28.0    power:  20,   rssi l/r:  218/214 
  28.1    power:  20,   rssi l/r:  218/214 
  28.2    power:  20,   rssi l/r:  218/214 
  28.3    power:  20,   rssi l/r:  218/214 
  28.4    power:  20,   rssi l/r:  218/214 
  28.5    power:  20,   rssi l/r:  218/214 
  28.6    power:  20,   rssi l/r:  218/214 
  28.7    power:  20,   rssi l/r:  218/214 
  28.8    power:  20,   rssi l/r:  218/214 
  28.9    power:  20,   rssi l/r:  218/214 
  28.0    power:  30,   rssi l/r

KeyboardInterrupt: 

## Get Tag's Transmit Power value

In [ ]:
STOP

In [48]:
def tag_get_power():
    name = TagName([TagTlv(tlv_types.NODE_ID, -1),
                            TagTlv('tag'),
                            TagTlv('.test'),
                            TagTlv('tx_pwr'),
                             ])
    req_obj = TagGet(name)
    req_msg = req_obj.build()
    sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)
    rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, 3)
    if rsp_msg:
        #        print(hexlify(rsp_msg))
        rsp_obj = TagMessage(rsp_msg)
        #        print(rsp_obj.header.options.param.error_code)
        #        print(rsp_obj.payload)
        print(rsp_obj.header, rsp_obj.name)
        if rsp_obj.payload:
            print(rsp_obj.payload)
        return rsp_obj.payload[0].value(), rssi, sstatus, rstatus
    return None, None, sstatus, rstatus

In [49]:
tag_get_power()

Container:
    frame_length = 34
    options = Container:
        response = True
        version = 1
        tlv_payload = 'TLV_LIST'
        message_type = 'GET'
        param = Container:
            hop_count = 0
            error_code = 'OK'
    name_length = 28 [(tlv_types.NODE_ID, ffffffffffff), (tlv_types.STRING, tag), (tlv_types.STRING, .test), (tlv_types.STRING, tx_pwr)]
[(tlv_types.INTEGER, 10)]


(10,
 218,
 [datetime.datetime(2018, 3, 28, 22, 34, 56, 514433),
  'c',
  '64',
  ',',
  '32',
  'w',
  Container({'CHIP_READY': True, 'STATE_CHANGE': True, 'WUT': False, 'CMD_ERROR': False, 'FIFO_UNDERFLOW_OVERFLOW_ERROR': False, 'LOW_BATT': False, 'CAL': False}),
  Container({'INVALID_SYNC': False, 'SYNC_DETECT': False, 'POSTAMBLE_DETECT': False, 'PREAMBLE_DETECT': True, 'INVALID_PREAMBLE': False, 'RSSI': False, 'RSSI_JUMP': False}),
  datetime.datetime(2018, 3, 28, 22, 34, 56, 555059),
  'w',
  Container({'CHIP_READY': True, 'STATE_CHANGE': False, 'WUT': False, 'CMD_ERROR': False, 'FIFO_UNDERFLOW_OVERFLOW_ERROR': False, 'LOW_BATT': False, 'CAL': False}),
  Container({'INVALID_SYNC': False, 'SYNC_DETECT': False, 'POSTAMBLE_DETECT': False, 'PREAMBLE_DETECT': True, 'INVALID_PREAMBLE': False, 'RSSI': False, 'RSSI_JUMP': False}),
  datetime.datetime(2018, 3, 28, 22, 34, 56, 566772),
  'w',
  Container({'CHIP_READY': True, 'STATE_CHANGE': False, 'WUT': False, 'CMD_ERROR': False, 'FIFO_UND

In [ ]:
radio.spi.trace.display(radio.spi.trace.filter(count=-100))

## Set Tag's Transmit Power value

In [ ]:
STOP

In [53]:
def tag_set_power(power):
    name = TagName([TagTlv(tlv_types.NODE_ID, -1),
                            TagTlv('tag'),
                            TagTlv('.test'),
                            TagTlv('tx_pwr'),
                             ])
    req_obj = TagPut(name, pl=TagTlvList([TagTlv(tlv_types.INTEGER, power)]))
    req_msg = req_obj.build()
    sstatus = radio_send_msg(radio, req_msg, RADIO_POWER)
    rsp_msg, rssi, rstatus = radio_receive_msg(radio, MAX_RECV, 3)
    if rsp_msg:
        #        print(hexlify(rsp_msg))
        rsp_obj = TagMessage(rsp_msg)
        #        print(rsp_obj.header.options.param.error_code)
        #        print(rsp_obj.payload)
        #print(rsp_obj.header, rsp_obj.name)
        #if rsp_obj.payload:
        #    print(rsp_obj.payload)
        return rsp_obj.payload, rssi, sstatus, rstatus
    return None, None, sstatus, rstatus

In [ ]:
for z in range(10000):
    for TAG_RADIO_POWER in range(1,130,10):
        for i in range(6):
            _, rssi, _, _ = tag_set_power(TAG_RADIO_POWER)
            tag_rssi, rssi, _, _ = tag_get_rssi()
            if rssi == -1:
                break
            print('{:>4}.{:<4} power: {:3},   rssi l/r:  {:4>}/{:<4}'.format(z,i, TAG_RADIO_POWER, rssi, tag_rssi))

   0.0    power:   1,   rssi l/r:  174/201 
   0.1    power:   1,   rssi l/r:  174/202 
   0.2    power:   1,   rssi l/r:  174/201 
   0.3    power:   1,   rssi l/r:  173/201 
   0.4    power:   1,   rssi l/r:  174/201 
   0.5    power:   1,   rssi l/r:  173/201 
   0.0    power:  11,   rssi l/r:  221/201 
   0.1    power:  11,   rssi l/r:  221/201 
   0.2    power:  11,   rssi l/r:  220/201 
   0.3    power:  11,   rssi l/r:  221/201 
   0.4    power:  11,   rssi l/r:  221/201 
   0.5    power:  11,   rssi l/r:  221/201 
   0.0    power:  21,   rssi l/r:  231/201 
   0.1    power:  21,   rssi l/r:  231/201 
   0.2    power:  21,   rssi l/r:  231/201 
   0.3    power:  21,   rssi l/r:  231/201 
   0.4    power:  21,   rssi l/r:  231/201 
   0.5    power:  21,   rssi l/r:  232/201 
   0.0    power:  31,   rssi l/r:  237/201 
   0.1    power:  31,   rssi l/r:  236/201 
   0.2    power:  31,   rssi l/r:  237/201 
   0.3    power:  31,   rssi l/r:  237/201 
   0.4    power:  31,   rssi l/r

   2.1    power:  51,   rssi l/r:  228/187 
   2.2    power:  51,   rssi l/r:  227/186 
   2.3    power:  51,   rssi l/r:  228/186 
   2.4    power:  51,   rssi l/r:  228/186 
   2.5    power:  51,   rssi l/r:  227/187 
   2.0    power:  61,   rssi l/r:  229/186 
   2.1    power:  61,   rssi l/r:  228/186 
   2.2    power:  61,   rssi l/r:  228/186 
   2.3    power:  61,   rssi l/r:  228/187 
   2.4    power:  61,   rssi l/r:  228/186 
   2.5    power:  61,   rssi l/r:  228/187 
   2.0    power:  71,   rssi l/r:  229/187 
   2.1    power:  71,   rssi l/r:  228/186 
   2.2    power:  71,   rssi l/r:  229/188 
   2.3    power:  71,   rssi l/r:  228/186 
   2.4    power:  71,   rssi l/r:  229/187 
   2.5    power:  71,   rssi l/r:  228/186 
   2.0    power:  81,   rssi l/r:  229/188 
   2.1    power:  81,   rssi l/r:  228/186 
   2.2    power:  81,   rssi l/r:  229/187 
   2.3    power:  81,   rssi l/r:  228/187 
   2.4    power:  81,   rssi l/r:  229/187 
   2.5    power:  81,   rssi l/r

   4.2    power: 101,   rssi l/r:  234/190 
   4.3    power: 101,   rssi l/r:  236/191 
   4.4    power: 101,   rssi l/r:  233/193 
   4.5    power: 101,   rssi l/r:  235/193 
   4.0    power: 111,   rssi l/r:  236/192 
   4.1    power: 111,   rssi l/r:  235/191 
   4.2    power: 111,   rssi l/r:  233/193 
   4.3    power: 111,   rssi l/r:  236/190 
   4.4    power: 111,   rssi l/r:  236/192 
   4.5    power: 111,   rssi l/r:  233/191 
   4.0    power: 121,   rssi l/r:  236/192 
   4.1    power: 121,   rssi l/r:  236/193 
   4.2    power: 121,   rssi l/r:  234/190 
   4.3    power: 121,   rssi l/r:  236/192 
   4.4    power: 121,   rssi l/r:  235/192 
   4.5    power: 121,   rssi l/r:  233/193 
   5.0    power:   1,   rssi l/r:  167/190 
   5.1    power:   1,   rssi l/r:  167/192 
   5.2    power:   1,   rssi l/r:  165/190 
   5.3    power:   1,   rssi l/r:  166/193 
   5.4    power:   1,   rssi l/r:  167/191 
   5.5    power:   1,   rssi l/r:  167/190 
   5.0    power:  11,   rssi l/r

   7.3    power:  21,   rssi l/r:  234/203 
   7.4    power:  21,   rssi l/r:  234/203 
   7.5    power:  21,   rssi l/r:  234/203 
   7.0    power:  31,   rssi l/r:  239/203 
   7.1    power:  31,   rssi l/r:  239/203 
   7.2    power:  31,   rssi l/r:  240/203 
   7.3    power:  31,   rssi l/r:  240/203 
   7.4    power:  31,   rssi l/r:  240/203 
   7.5    power:  31,   rssi l/r:  240/204 
   7.0    power:  41,   rssi l/r:  241/203 
   7.1    power:  41,   rssi l/r:  240/203 
   7.2    power:  41,   rssi l/r:  241/203 
   7.3    power:  41,   rssi l/r:  241/203 
   7.4    power:  41,   rssi l/r:  241/205 
   7.5    power:  41,   rssi l/r:  241/203 
   7.0    power:  51,   rssi l/r:  242/203 
   7.1    power:  51,   rssi l/r:  242/203 
   7.2    power:  51,   rssi l/r:  241/203 
   7.3    power:  51,   rssi l/r:  242/203 
   7.4    power:  51,   rssi l/r:  242/203 
   7.5    power:  51,   rssi l/r:  242/203 
   7.0    power:  61,   rssi l/r:  242/203 
   7.1    power:  61,   rssi l/r

   9.4    power:  71,   rssi l/r:  242/204 
   9.5    power:  71,   rssi l/r:  243/203 
   9.0    power:  81,   rssi l/r:  243/203 
   9.1    power:  81,   rssi l/r:  243/203 
   9.2    power:  81,   rssi l/r:  243/203 
   9.3    power:  81,   rssi l/r:  243/203 
   9.4    power:  81,   rssi l/r:  242/203 
   9.5    power:  81,   rssi l/r:  243/203 
   9.0    power:  91,   rssi l/r:  243/203 
   9.1    power:  91,   rssi l/r:  243/203 
   9.2    power:  91,   rssi l/r:  243/203 
   9.3    power:  91,   rssi l/r:  243/204 
   9.4    power:  91,   rssi l/r:  243/203 
   9.5    power:  91,   rssi l/r:  243/203 
   9.0    power: 101,   rssi l/r:  243/203 
   9.1    power: 101,   rssi l/r:  243/203 
   9.2    power: 101,   rssi l/r:  243/203 
   9.3    power: 101,   rssi l/r:  243/203 
   9.4    power: 101,   rssi l/r:  243/203 
   9.5    power: 101,   rssi l/r:  243/203 
   9.0    power: 111,   rssi l/r:  243/203 
   9.1    power: 111,   rssi l/r:  243/203 
   9.2    power: 111,   rssi l/r

...

  67.4    power:  11,   rssi l/r:  222/203 
  67.5    power:  11,   rssi l/r:  222/203 
  67.0    power:  21,   rssi l/r:  234/203 
  67.1    power:  21,   rssi l/r:  234/203 
  67.2    power:  21,   rssi l/r:  234/203 
  67.3    power:  21,   rssi l/r:  234/203 
  67.4    power:  21,   rssi l/r:  234/203 
  67.5    power:  21,   rssi l/r:  234/203 
  67.0    power:  31,   rssi l/r:  239/203 
  67.1    power:  31,   rssi l/r:  239/203 
  67.2    power:  31,   rssi l/r:  239/203 
  67.3    power:  31,   rssi l/r:  239/203 
  67.4    power:  31,   rssi l/r:  239/203 
  67.5    power:  31,   rssi l/r:  239/203 
  67.0    power:  41,   rssi l/r:  241/203 
  67.1    power:  41,   rssi l/r:  241/203 
  67.2    power:  41,   rssi l/r:  241/203 
  67.3    power:  41,   rssi l/r:  241/203 
  67.4    power:  41,   rssi l/r:  241/203 
  67.5    power:  41,   rssi l/r:  241/203 
  67.0    power:  51,   rssi l/r:  242/203 
  67.1    power:  51,   rssi l/r:  242/203 
  67.2    power:  51,   rssi l/r

  69.5    power:  61,   rssi l/r:  242/203 
  69.0    power:  71,   rssi l/r:  242/203 
  69.1    power:  71,   rssi l/r:  242/203 
  69.2    power:  71,   rssi l/r:  242/203 
  69.3    power:  71,   rssi l/r:  242/203 
  69.4    power:  71,   rssi l/r:  242/203 
  69.5    power:  71,   rssi l/r:  242/203 
  69.0    power:  81,   rssi l/r:  242/203 
  69.1    power:  81,   rssi l/r:  242/203 
  69.2    power:  81,   rssi l/r:  242/203 
  69.3    power:  81,   rssi l/r:  242/203 
  69.4    power:  81,   rssi l/r:  242/203 
  69.5    power:  81,   rssi l/r:  242/203 
  69.0    power:  91,   rssi l/r:  242/203 
  69.1    power:  91,   rssi l/r:  242/203 
  69.2    power:  91,   rssi l/r:  242/203 
  69.3    power:  91,   rssi l/r:  243/203 
  69.4    power:  91,   rssi l/r:  242/203 
  69.5    power:  91,   rssi l/r:  242/203 
  69.0    power: 101,   rssi l/r:  243/203 
  69.1    power: 101,   rssi l/r:  242/203 
  69.2    power: 101,   rssi l/r:  243/203 
  69.3    power: 101,   rssi l/r

  71.0    power: 121,   rssi l/r:  243/203 
  71.1    power: 121,   rssi l/r:  243/203 
  71.2    power: 121,   rssi l/r:  243/203 
  71.3    power: 121,   rssi l/r:  243/203 
  71.4    power: 121,   rssi l/r:  243/203 
  71.5    power: 121,   rssi l/r:  243/203 
  72.0    power:   1,   rssi l/r:  175/203 
  72.1    power:   1,   rssi l/r:  175/203 
  72.2    power:   1,   rssi l/r:  175/203 
  72.3    power:   1,   rssi l/r:  175/203 
  72.4    power:   1,   rssi l/r:  175/203 
  72.5    power:   1,   rssi l/r:  175/203 
  72.0    power:  11,   rssi l/r:  222/203 
  72.1    power:  11,   rssi l/r:  222/203 
  72.2    power:  11,   rssi l/r:  222/203 
  72.3    power:  11,   rssi l/r:  222/203 
  72.4    power:  11,   rssi l/r:  222/203 
  72.5    power:  11,   rssi l/r:  222/203 
  72.0    power:  21,   rssi l/r:  234/203 
  72.1    power:  21,   rssi l/r:  234/203 
  72.2    power:  21,   rssi l/r:  234/203 
  72.3    power:  21,   rssi l/r:  234/203 
  72.4    power:  21,   rssi l/r

  74.1    power:  41,   rssi l/r:  241/203 
  74.2    power:  41,   rssi l/r:  241/203 
  74.3    power:  41,   rssi l/r:  241/203 
  74.4    power:  41,   rssi l/r:  241/203 
  74.5    power:  41,   rssi l/r:  241/203 
  74.0    power:  51,   rssi l/r:  242/203 
  74.1    power:  51,   rssi l/r:  242/203 
  74.2    power:  51,   rssi l/r:  242/203 
  74.3    power:  51,   rssi l/r:  242/203 
  74.4    power:  51,   rssi l/r:  242/203 
  74.5    power:  51,   rssi l/r:  242/203 
  74.0    power:  61,   rssi l/r:  242/203 
  74.1    power:  61,   rssi l/r:  242/203 
  74.2    power:  61,   rssi l/r:  242/203 
  74.3    power:  61,   rssi l/r:  242/203 
  74.4    power:  61,   rssi l/r:  242/203 
  74.5    power:  61,   rssi l/r:  242/203 
  74.0    power:  71,   rssi l/r:  242/203 
  74.1    power:  71,   rssi l/r:  242/203 
  74.2    power:  71,   rssi l/r:  242/203 
  74.3    power:  71,   rssi l/r:  242/203 
  74.4    power:  71,   rssi l/r:  242/203 
  74.5    power:  71,   rssi l/r

  76.2    power:  91,   rssi l/r:  243/203 
  76.3    power:  91,   rssi l/r:  242/203 
  76.4    power:  91,   rssi l/r:  242/203 
  76.5    power:  91,   rssi l/r:  242/203 
  76.0    power: 101,   rssi l/r:  243/203 
  76.1    power: 101,   rssi l/r:  243/203 
  76.2    power: 101,   rssi l/r:  243/203 
  76.3    power: 101,   rssi l/r:  243/203 
  76.4    power: 101,   rssi l/r:  243/203 
  76.5    power: 101,   rssi l/r:  243/203 
  76.0    power: 111,   rssi l/r:  243/203 
  76.1    power: 111,   rssi l/r:  243/203 
  76.2    power: 111,   rssi l/r:  243/203 
  76.3    power: 111,   rssi l/r:  243/203 
  76.4    power: 111,   rssi l/r:  243/203 
  76.5    power: 111,   rssi l/r:  243/203 
  76.0    power: 121,   rssi l/r:  243/203 
  76.1    power: 121,   rssi l/r:  243/203 
  76.2    power: 121,   rssi l/r:  243/203 
  76.3    power: 121,   rssi l/r:  243/203 
  76.4    power: 121,   rssi l/r:  243/203 
  76.5    power: 121,   rssi l/r:  243/203 
  77.0    power:   1,   rssi l/r

In [54]:
TAG_RADIO_POWER=10

In [47]:
tag_set_power(TAG_RADIO_POWER)

Container:
    frame_length = 31
    options = Container:
        response = True
        version = 1
        tlv_payload = 'TLV_LIST'
        message_type = 'PUT'
        param = Container:
            hop_count = 0
            error_code = 'OK'
    name_length = 28 [(tlv_types.NODE_ID, ffffffffffff), (tlv_types.STRING, tag), (tlv_types.STRING, .test), (tlv_types.STRING, tx_pwr)]


(None,
 218,
 [datetime.datetime(2018, 3, 28, 22, 34, 12, 659496),
  'c',
  '64',
  ',',
  '35',
  'w',
  Container({'CHIP_READY': True, 'STATE_CHANGE': True, 'WUT': False, 'CMD_ERROR': False, 'FIFO_UNDERFLOW_OVERFLOW_ERROR': False, 'LOW_BATT': False, 'CAL': False}),
  Container({'INVALID_SYNC': False, 'SYNC_DETECT': False, 'POSTAMBLE_DETECT': False, 'PREAMBLE_DETECT': True, 'INVALID_PREAMBLE': False, 'RSSI': False, 'RSSI_JUMP': False}),
  datetime.datetime(2018, 3, 28, 22, 34, 12, 701170),
  'w',
  Container({'CHIP_READY': True, 'STATE_CHANGE': False, 'WUT': False, 'CMD_ERROR': False, 'FIFO_UNDERFLOW_OVERFLOW_ERROR': False, 'LOW_BATT': False, 'CAL': False}),
  Container({'INVALID_SYNC': False, 'SYNC_DETECT': False, 'POSTAMBLE_DETECT': False, 'PREAMBLE_DETECT': True, 'INVALID_PREAMBLE': False, 'RSSI': False, 'RSSI_JUMP': False}),
  datetime.datetime(2018, 3, 28, 22, 34, 12, 712937),
  'w',
  Container({'CHIP_READY': True, 'STATE_CHANGE': False, 'WUT': False, 'CMD_ERROR': False, 'FIFO_U

In [36]:
radio.spi.trace.display(radio.spi.trace.filter(count=-100))

[['2018.03.28 22:28:47.652884',
  ' -4.629907',
  'RADIO_PEND',
  '  radio_send_msg:372 int_status:314',
  39020,
  ' int_status_rsp_s  modem_pend:  PREAMBLE_DETECT chip_pend:  CHIP_READY ph_status:  PACKET_RX TX_FIFO_ALMOST_EMPTY modem_status:  RSSI chip_status:  STATE_CHANGE CHIP_READY'],
 ['2018.03.28 22:28:47.657341',
  ' -4.625449',
  'RADIO_CMD',
  '  get_clear_interrupts:425 clear_interrupts:244',
  39021,
  ' get_clear_int_cmd_s  ph_pend:  CRC_ERROR modem_pend:  POSTAMBLE_DETECT INVALID_SYNC PREAMBLE_DETECT chip_pend:  CMD_ERROR CHIP_READY'],
 ['2018.03.28 22:28:47.657586',
  ' -4.625205',
  'RADIO_PEND',
  '  int_status:314 get_clear_interrupts:425',
  39022,
  ' get_clear_int_cmd_s  ph_pend:  CRC_ERROR modem_pend:  POSTAMBLE_DETECT INVALID_SYNC PREAMBLE_DETECT chip_pend:  CMD_ERROR CHIP_READY'],
 ['2018.03.28 22:28:47.658301',
  ' -4.624490',
  'RADIO_RSP',
  '  int_status:314 get_clear_interrupts:426',
  39023,
  ' int_status_rsp_s  ph_pend:  PACKET_SENT modem_pend:  PREAMBL

### Write Zeros

In [ ]:
STOP

In [ ]:
#<node_id>   "tag"  "test"   "zero"   "byte"
def read_test_bytes(testfile, pos, num):
    zero_byte_name = TagName([TagTlv(tlv_types.NODE_ID, -1),
                            TagTlv('tag'),
                            TagTlv('.test'),
                            TagTlv(testfile),
                            TagTlv('byte'),
                            TagTlv(tlv_types.OFFSET, pos),
                            TagTlv(tlv_types.SIZE, num),
                             ])
    get_zero_byte = TagGet(zero_byte_name)
#    print(get_gps_xyz.name)
    req_msg = get_zero_byte.build()
    radio_send_msg(radio, req_msg, RADIO_POWER);
    rsp_msg, rssi, status = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
    if rsp_msg:
#        print(hexlify(rsp_msg))
        rsp_obj = TagMessage(rsp_msg)
#        print(rsp_obj.header.options.param.error_code)
#        print(rsp_obj.payload)
        if rsp_obj.payload:
            error, offset, amt, block = payload2values(rsp_obj.payload,
                                  [tlv_types.ERROR,
                                   tlv_types.OFFSET,
                                   tlv_types.SIZE,
                                   tlv_types.BLOCK,
                                  ])
            seta = set(block)
            # print(seta)
            if (len(seta) > 1):
                print('check error', seta, amt, hexlify(block))
                amt = 0
            return error, offset, amt, block
        else:
            print("{}".format(rsp_obj.header.options.param.error_code))
#    else:
#        print('TIMEOUT')
    return None

In [ ]:
start = datetime.datetime.now()
for x in range(1,1000):
    for y in range(1,10):
        sleep(SHORT_DELAY)
        return_values = read_test_bytes('zero', 1, 1)
        if return_values:
            error, offset, amt, block = return_values
            # print('error: {}'.format(error))
            if error and error is not tlv_errors.SUCCESS:
                print("\r{} {}".format(datetime.now() - start, error), end="")
                continue
            print('got bytes, len: {}, count: {}, offset: {}'.format(len(block),
                                                                    amt,
                                                                    offset))
        else:
            print('timeout')
            
print("\ntime:{}".format(datetime.now() - start))

## Listen for packets

In [ ]:
for i in range(200000):
    rsp_msg, rssi, status = radio_receive_msg(radio, MAX_RECV, 3000)
    if rsp_msg:
        print(datetime.datetime.now(), rssi, len(rsp_msg), hexlify(rsp_msg))
        #rsp_obj = TagMessage(rsp_msg)
        #        print(rsp_obj.header.options.param.error_code)
        #        print(rsp_obj.payload)
        #if rsp_obj.payload:
        #    print(rsp_obj.header, rsp_obj.name, rsp_obj.payload)

## Get Chip Status

In [ ]:
STOP

In [ ]:
print(radio.get_chip_status())

## Interactive Group Properties

In [ ]:
STOP

In [ ]:
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

## Interactive  Command Status Responses

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

In [ ]:
from datetime import datetime
datetime.now()